In [9]:
# imports
import csv
import urllib
import io
import numpy as np
import pandas as pd
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160326.txt')

In [10]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248


In [11]:
#Fix last columns
data.rename(columns={data.columns[-1]: data.columns[-1].replace(' ','')}, inplace=True)
data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [ ]:
#data['Delta'] = abs((data['ENTRIES'] - data['ENTRIES'].shift(-1)) + (data['EXITS'] - data['EXITS'].shift(-1)))

In [ ]:
#test = data.sort_values(['Delta'])


In [ ]:
#data[['TIME','DATE']] = pd.to_datetime(data[['TIME','DATE']])


In [ ]:
#df_night = data[data['TIME' ==  ]

In [ ]:
# Aggregate
mask_key = data.ix[:,0]
for r in range(data.shape[0]):
    k = tuple(data.ix[1:2,4:])
    if k in fill.keys():
        fill[k] += 1
    else:
        fill[k] = 1
fill

In [ ]:
import numpy as np
tuple(data.ix[1:2,4:])
fill

In [24]:
mask_key = data.ix[:,0]

In [20]:
np.array('')

array('', 
      dtype='<U1')

In [25]:
mask_key

0          A002
1          A002
2          A002
3          A002
4          A002
5          A002
6          A002
7          A002
8          A002
9          A002
10         A002
11         A002
12         A002
13         A002
14         A002
15         A002
16         A002
17         A002
18         A002
19         A002
20         A002
21         A002
22         A002
23         A002
24         A002
25         A002
26         A002
27         A002
28         A002
29         A002
          ...  
192916    TRAM2
192917    TRAM2
192918    TRAM2
192919    TRAM2
192920    TRAM2
192921    TRAM2
192922    TRAM2
192923    TRAM2
192924    TRAM2
192925    TRAM2
192926    TRAM2
192927    TRAM2
192928    TRAM2
192929    TRAM2
192930    TRAM2
192931    TRAM2
192932    TRAM2
192933    TRAM2
192934    TRAM2
192935    TRAM2
192936    TRAM2
192937    TRAM2
192938    TRAM2
192939    TRAM2
192940    TRAM2
192941    TRAM2
192942    TRAM2
192943    TRAM2
192944    TRAM2
192945    TRAM2
Name: C/A, dtype: object